In [1]:
import os
import sys
current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
sys.path.append(parent_dir)
from utils.env_util import *
from langgraph_utils.common_util import gen_mermaid
from pydantic import BaseModel
from typing_extensions import Literal
from langgraph.graph import StateGraph, START, END, MessagesState
from langgraph.checkpoint.memory import MemorySaver
from langgraph.types import Command, interrupt
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool


# 定义参数模型
class WeatherParams(BaseModel):
    city: str

@tool(args_schema=WeatherParams)
def weather_search(city: str) -> str:
    """Search for the weather"""
    print("-" * 80)
    print(f"⚙️ 正在查询天气: {city}")
    print("-" * 80)
    return "晴天" if ("北京" in city) else "雨天"

model = ChatOpenAI(
    openai_api_key=get_openai_api_key(),
    model_name=get_default_model(),
    base_url=get_openai_base_url(),
).bind_tools(
    [weather_search]
)

class State(MessagesState):
    """Simple state."""


def call_llm(state):
    return {"messages": [model.invoke(state["messages"])]}


def human_review_node(state) -> Command[Literal["call_llm", "run_tool"]]:
    last_message_from_llm = state["messages"][-1]
    tool_call = last_message_from_llm.tool_calls[-1]

    # 这是我们将通过 Command(resume=<human_review>)提供的值
    human_review = interrupt(
        {
            "question": "请检查结果是否正确",
            # Surface tool calls for review
            "tool_call": tool_call,
        }
    )

    review_action = human_review["action"]
    review_data = human_review.get("data")

    # 审批通过，同意调用工具
    if review_action == "continue":
        print("⚙️ 同意调用工具")
        return Command(goto="run_tool")

    # update the AI message AND call tools
    elif review_action == "update":
        updated_message = {
            "role": "ai",
            "content": last_message_from_llm.content,
            "tool_calls": [
                {
                    "id": tool_call["id"],
                    "name": tool_call["name"],
                    # This the update provided by the human
                    "args": review_data,
                }
            ],
            # id 需要与替换的消息相同！否则，它将显示为单独的消息
            "id": last_message_from_llm.id,
        }
        print("⚙️ 消息已更新，同意调用工具")
        return Command(goto="run_tool", update={"messages": [updated_message]})

    # 向LLM提供反馈
    elif review_action == "feedback":
        # 因为在 route_after_llm -> human_review_node 这条路径中，LLM 会调用工具
        # 因此需要将反馈消息封装成 ToolMessage 添加到消息列表中
        # 因为存在 tool calls 的 AIMessage 后面必须有 ToolMessage
        tool_message = {
            "role": "tool",
            # This is our natural language feedback
            "content": review_data,
            "name": tool_call["name"],
            "tool_call_id": tool_call["id"],
        }
        print("⚙️ 审核未通过，返回 LLM")
        return Command(goto="call_llm", update={"messages": [tool_message]})


def run_tool(state):
    """ 调用工具 """
    new_messages = []
    tools = {"weather_search": weather_search}
    tool_calls = state["messages"][-1].tool_calls
    for tool_call in tool_calls:
        tool = tools[tool_call["name"]]
        print(f"⚙️ 开始调用工具: {tool_call['name']}")
        result = tool.invoke(tool_call["args"])
        new_messages.append(
            {
                "role": "tool",
                "name": tool_call["name"],
                "content": result,
                "tool_call_id": tool_call["id"],
            }
        )
    return {"messages": new_messages}


def route_after_llm(state) -> Literal[END, "human_review_node"]:
    """
        如果 LLM 的响应需要调用外部工具(如查询数据库、调用 API),则会在 tool_calls 中记录这些调用。
        若 tool_calls 为空: 表示 LLM 无需调用任何工具，可以直接返回最终结果 → 流程结束(END)
    """
    if len(state["messages"][-1].tool_calls) == 0:
        return END
    else:
        return "human_review_node"


builder = StateGraph(State)
builder.add_node(call_llm)
builder.add_node(run_tool)
builder.add_node(human_review_node)
builder.add_edge(START, "call_llm")
builder.add_conditional_edges("call_llm", route_after_llm)
builder.add_edge("run_tool", "call_llm")

memory = MemorySaver()
graph = builder.compile(checkpointer=memory)

✔️ OPENAI_API_KEY: sk-hybehttizlquaobtbilikijqmuuyzxizjhkfqqlpkkvcvojw
✔️ DEFAULT_MODEL: Qwen/QwQ-32B
✔️ OPENAI_BASE_URL: https://api.siliconflow.cn/v1


In [2]:
gen_mermaid(graph, "human_review.mmd")

生成 mermaid 文件 /Users/yuki/codes/pythonProject/Agent/langgraph_demo/resources/human_review.mmd
✏️ 已生成 mermaid 文件 /Users/yuki/codes/pythonProject/Agent/langgraph_demo/resources/human_review.mmd


## 1. 普通的调用（没有调用工具，无需人工审核）

In [26]:
# 一个无需审批的示例（因为没有调用任何工具）
initial_input = {"messages": [{"role": "user", "content": "你好"}]}
thread = {"configurable": {"thread_id": "1"}}
# 运行图直到第一个中断
for event in graph.stream(initial_input, thread, stream_mode="updates"):
    print(event)
    print("\n")

{'call_llm': {'messages': [AIMessage(content='你好！有什么可以帮你的吗？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 124, 'prompt_tokens': 151, 'total_tokens': 275, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 115, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': None}, 'model_name': 'Qwen/QwQ-32B', 'system_fingerprint': '', 'id': '019666886673cdea77e6e523da85f339', 'finish_reason': 'stop', 'logprobs': None}, id='run-4a26797b-faf0-4e7d-9d38-3eda13fd627f-0', usage_metadata={'input_tokens': 151, 'output_tokens': 124, 'total_tokens': 275, 'input_token_details': {}, 'output_token_details': {'reasoning': 115}})]}}




In [27]:
# {
#     'call_llm': {
#         'messages': [
#             AIMessage(
#                 content='你好！有什么我可以帮你的吗？', 
#                 additional_kwargs={'refusal': None}, 
#                 response_metadata={'token_usage': {'completion_tokens': 218, 'prompt_tokens': 151, 'total_tokens': 369, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 209, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': None}, 'model_name': 'Qwen/QwQ-32B', 'system_fingerprint': '', 'id': '0196666a22287404c2b42f3bedd583f9', 'finish_reason': 'stop', 'logprobs': None}, 
#                 id='run-a33ba9bd-255f-41da-abd0-f9a3a20478c9-0', 
#                 usage_metadata={'input_tokens': 151, 'output_tokens': 218, 'total_tokens': 369, 'input_token_details': {}, 'output_token_details': {'reasoning': 209}}
#             )
#         ]
#     }
# }

## 2. 人工批准工具调用

In [33]:
# 批准一次工具调用
initial_input = {"messages": [{"role": "user", "content": "上海天气怎么样？"}]}
thread = {"configurable": {"thread_id": "2"}}
for event in graph.stream(initial_input, thread, stream_mode="updates"):
    print(event)
    print("\n")
print("⚙️ 等待执行", graph.get_state(thread).next)

{'call_llm': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '0196668d453c72e072eb123b2de72ca1', 'function': {'arguments': '{"city": "上海"}', 'name': 'weather_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 143, 'prompt_tokens': 154, 'total_tokens': 297, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 123, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': None}, 'model_name': 'Qwen/QwQ-32B', 'system_fingerprint': '', 'id': '0196668d26dac2344d1f1b0293f6e5c9', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-1fb021dc-46a4-48e7-a47b-88c302be6a2e-0', tool_calls=[{'name': 'weather_search', 'args': {'city': '上海'}, 'id': '0196668d453c72e072eb123b2de72ca1', 'type': 'tool_call'}], usage_metadata={'input_tokens': 154, 'output_tokens': 143, 'total_tokens': 297, 'input_token_details': {}, 'output_token_details': {'reasoning': 123}})]

In [29]:
# {
#     'call_llm': {
#         'messages': [
#             AIMessage(
#                 content='', 
#                 additional_kwargs={
#                     'tool_calls': [{
#                             'id': '0196666b07dd2443fcd09d99f2da6bd1', 
#                             'function': {'arguments': '{"city": "上海"}', 'name': 'weather_search'}, 
#                             'type': 'function'
#                         }], 
#                     'refusal': None
#                 }, 
#                 response_metadata={'token_usage': {'completion_tokens': 133, 'prompt_tokens': 154, 'total_tokens': 287, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 113, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': None}, 'model_name': 'Qwen/QwQ-32B', 'system_fingerprint': '', 'id': '0196666af547108bab2e3453ec3c6ecb', 'finish_reason': 'tool_calls', 'logprobs': None}, 
#                 id='run-d44f5bca-f98a-4394-a88c-f2923ca382c6-0', 
#                 tool_calls=[
#                     {'name': 'weather_search', 'args': {'city': '上海'}, 'id': '0196666b07dd2443fcd09d99f2da6bd1', 'type': 'tool_call'}
#                 ], 
#                 usage_metadata={'input_tokens': 154, 'output_tokens': 133, 'total_tokens': 287, 'input_token_details': {}, 'output_token_details': {'reasoning': 113}}
#             )
#         ]
#     }
# }

In [30]:
# {
#     '__interrupt__': (
#         Interrupt(
#             value={
#                 'question': '请检查结果是否正确', 
#                 'tool_call': {
#                     'name': 'weather_search', 
#                     'args': {'city': '上海'}, 
#                     'id': '0196666b07dd2443fcd09d99f2da6bd1', 
#                     'type': 'tool_call'
#                 }
#             }, 
#             resumable=True, 
#             ns=['human_review_node:1188182a-a0e0-4b3f-b5e3-f8ef73e601cc']),
#     )
# }

In [34]:
for event in graph.stream(
    # 同意调用工具
    Command(resume={"action": "continue"}),
    thread,
    stream_mode="updates",
):
    print(event)
    print("\n")

{'messages': [HumanMessage(content='上海天气怎么样？', additional_kwargs={}, response_metadata={}, id='baf6f0c8-f918-40d1-91fe-26a61f08e1ec'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '0196668d453c72e072eb123b2de72ca1', 'function': {'arguments': '{"city": "上海"}', 'name': 'weather_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 143, 'prompt_tokens': 154, 'total_tokens': 297, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 123, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': None}, 'model_name': 'Qwen/QwQ-32B', 'system_fingerprint': '', 'id': '0196668d26dac2344d1f1b0293f6e5c9', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-1fb021dc-46a4-48e7-a47b-88c302be6a2e-0', tool_calls=[{'name': 'weather_search', 'args': {'city': '上海'}, 'id': '0196668d453c72e072eb123b2de72ca1', 'type': 'tool_call'}], usage_metadata={'input_tokens': 154, 'output

一次人工审核通过的工具调用的流程如下：
1. 用户输入：“上海天气怎么样？”，生成 `HumanMessage`
2. 从 `START` 节点流转到 `call_llm` 节点，调用 LLM，LLM 判断需要调用 `Tools`, 生成了带 `tool_call` 信息的 `AIMessage`
3. `route_after_llm` 把流程从 `call_llm` 节点路由到 `human_review_node` 节点，调用了 `interrupt()` 产生中断，人工介入后同意调用工具，从 `human_review_node` 节点流转到 `run_tool` 节点
4. `run_tool` 调用工具，生成 `ToolMessage`，内容存放在 `ToolMessage` 的 `content` 字段
5. 从 `run_tool` 节点流转到 `call_llm` 节点，调用 LLM，LLM 根据返回的结果判断已经不需要调用 `Tools` 了, 生成普通的 `AIMessage`
6. `route_after_llm` 把流程从 `call_llm` 节点路由到 `END`，流程结束

In [ ]:
# {
#     'messages': [
#         HumanMessage(
#             content='上海天气怎么样？', 
#             additional_kwargs={}, 
#             response_metadata={}, 
#             id='baf6f0c8-f918-40d1-91fe-26a61f08e1ec'
#         ), 
#         AIMessage(
#             content='', 
#             additional_kwargs={'tool_calls': [{'id': '0196668d453c72e072eb123b2de72ca1', 'function': {'arguments': '{"city": "上海"}', 'name': 'weather_search'}, 'type': 'function'}], 'refusal': None}, 
#             response_metadata={'token_usage': {'completion_tokens': 143, 'prompt_tokens': 154, 'total_tokens': 297, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 123, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': None}, 'model_name': 'Qwen/QwQ-32B', 'system_fingerprint': '', 'id': '0196668d26dac2344d1f1b0293f6e5c9', 'finish_reason': 'tool_calls', 'logprobs': None}, 
#             id='run-1fb021dc-46a4-48e7-a47b-88c302be6a2e-0', 
#             tool_calls=[{'name': 'weather_search', 'args': {'city': '上海'}, 'id': '0196668d453c72e072eb123b2de72ca1', 'type': 'tool_call'}], 
#             usage_metadata={'input_tokens': 154, 'output_tokens': 143, 'total_tokens': 297, 'input_token_details': {}, 'output_token_details': {'reasoning': 123}}
#         ), 
#         ToolMessage(
#             content='晴天!', 
#             name='weather_search', 
#             id='4cedfc74-1cf9-4e37-9c1f-d219eba7db11', 
#             tool_call_id='0196668d453c72e072eb123b2de72ca1'
#         ), 
#         AIMessage(
#             content='上海今天天气晴朗，是一个适合外出的好日子！如果有其他想知道的信息，随时告诉我哦～', 
#             additional_kwargs={'refusal': None}, 
#             response_metadata={'token_usage': {'completion_tokens': 222, 'prompt_tokens': 202, 'total_tokens': 424, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 200, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': None}, 'model_name': 'Qwen/QwQ-32B', 'system_fingerprint': '', 'id': '0196668d917e7d733c30f65a9f0bbedf', 'finish_reason': 'stop', 'logprobs': None}, 
#             id='run-65d71d0d-28cf-41df-9cbf-89f99151e892-0', 
#             usage_metadata={'input_tokens': 202, 'output_tokens': 222, 'total_tokens': 424, 'input_token_details': {}, 'output_token_details': {'reasoning': 200}}
#         )
#     ]
# }

## 3. 人工更新工具调用
可以更改某些参数（甚至更改所调用的工具！），然后执行该工具。

In [61]:
initial_input = {"messages": [{"role": "user", "content": "北京天气怎么样？"}]}
thread = {"configurable": {"thread_id": "3"}}
for event in graph.stream(initial_input, thread, stream_mode="updates"):
    print(event)
    print("\n")
print("⚙️ 等待执行", graph.get_state(thread).next)

{'call_llm': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '019666c85eabf135f179d0243c78c2ba', 'function': {'arguments': '{"city": "北京"}', 'name': 'weather_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 106, 'prompt_tokens': 154, 'total_tokens': 260, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 86, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': None}, 'model_name': 'Qwen/QwQ-32B', 'system_fingerprint': '', 'id': '019666c848a601cd3a392f1a0985c9de', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-e0a3a4b6-41c3-4cee-81ec-6767d767e049-0', tool_calls=[{'name': 'weather_search', 'args': {'city': '北京'}, 'id': '019666c85eabf135f179d0243c78c2ba', 'type': 'tool_call'}], usage_metadata={'input_tokens': 154, 'output_tokens': 106, 'total_tokens': 260, 'input_token_details': {}, 'output_token_details': {'reasoning': 86}})]}}

In [59]:
# 这里把在调用工具前把查询的参数 city 改成了杭州，最终查询到北京的天气变成雨天（实际是晴天）
for event in graph.stream(
    # Command 的 resume 参数必须包含 action 和 data 两个字段（action='continue'时可以没有 data）
    Command(resume={"action": "update", "data": {"city": "杭州"}}),
    thread,
    stream_mode="updates",
):
    print(event)
    print("\n")

⚙️ 消息已更新，同意调用工具
{'human_review_node': {'messages': [{'role': 'ai', 'content': '', 'tool_calls': [{'id': '019666c5e1a2be3462373931a820a9cf', 'name': 'weather_search', 'args': {'city': '杭州'}}], 'id': 'run-fa4a896d-cdb7-4d68-90b6-cbabba11ec77-0'}]}}


⚙️ 开始调用工具: weather_search
--------------------------------------------------------------------------------
⚙️ 正在查询天气: 杭州
--------------------------------------------------------------------------------
{'run_tool': {'messages': [{'role': 'tool', 'name': 'weather_search', 'content': '雨天', 'tool_call_id': '019666c5e1a2be3462373931a820a9cf'}]}}


{'call_llm': {'messages': [AIMessage(content='北京今天是雨天，请记得携带雨具，出行注意交通安全。需要查询其他城市天气或了解更详细信息吗？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 667, 'prompt_tokens': 200, 'total_tokens': 867, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 639, 'rejected_prediction_tokens': None}, 'prompt_tokens_detai

## 4. 对工具调用给出反馈
有时，你可能不想执行工具调用，但也不想人工手动修改工具调用。在这种情况下，从用户那里获取自然语言反馈可能会更好。你可以将此反馈作为工具调用的模拟结果插入。 

In [70]:
initial_input = {"messages": [{"role": "user", "content": "北京今天天气怎么样？"}]}
thread = {"configurable": {"thread_id": "4"}}
for event in graph.stream(initial_input, thread, stream_mode="updates"):
    print(event)
    print("\n")
print("⚙️ 等待执行", graph.get_state(thread).next)

{'call_llm': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '019666d89d0198036fc8e3bf34bca704', 'function': {'arguments': '{"city": "北京"}', 'name': 'weather_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 108, 'prompt_tokens': 155, 'total_tokens': 263, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 88, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': None}, 'model_name': 'Qwen/QwQ-32B', 'system_fingerprint': '', 'id': '019666d889766991ac8914b0a6f9f194', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-b5343477-dd2e-437a-bf1c-e44cf144acac-0', tool_calls=[{'name': 'weather_search', 'args': {'city': '北京'}, 'id': '019666d89d0198036fc8e3bf34bca704', 'type': 'tool_call'}], usage_metadata={'input_tokens': 155, 'output_tokens': 108, 'total_tokens': 263, 'input_token_details': {}, 'output_token_details': {'reasoning': 88}})]}}

In [71]:
for event in graph.stream(
    Command(
        resume={
            "action": "feedback",
            "data": "必须使用<城市,国家> 这种格式来调用天气查询工具，例如：北京,中国。注意查询参数使用使用 UTF-8 编码，不要使用 Unicode。",
        }
    ),
    thread,
    stream_mode="updates",
):
    print(event)
    print("\n")
print("⚙️ 等待执行", graph.get_state(thread).next)

⚙️ 审核未通过，返回 LLM
{'human_review_node': {'messages': [{'role': 'tool', 'content': '必须使用<城市,国家> 这种格式来调用天气查询工具，例如：北京,中国。注意查询参数使用使用 UTF-8 编码，不要使用 Unicode。', 'name': 'weather_search', 'tool_call_id': '019666d89d0198036fc8e3bf34bca704'}]}}


{'call_llm': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '019666d90a5564c2b314d48048d9f7b0', 'function': {'arguments': '{"city": "北京,中国"}', 'name': 'weather_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 642, 'prompt_tokens': 239, 'total_tokens': 881, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 620, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': None}, 'model_name': 'Qwen/QwQ-32B', 'system_fingerprint': '', 'id': '019666d8a8efb7323a2e83fa5ef4b6f9', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-129ced74-60ad-4d5a-afa9-57b7c92c4220-0', tool_calls=[{'name': 'weather_search', 'ar

In [72]:
# 审批通过
for event in graph.stream(
    Command(resume={"action": "continue"}), thread, stream_mode="updates"
):
    print(event)
    print("\n")

⚙️ 同意调用工具
{'human_review_node': None}


⚙️ 开始调用工具: weather_search
--------------------------------------------------------------------------------
⚙️ 正在查询天气: 北京,中国
--------------------------------------------------------------------------------
{'run_tool': {'messages': [{'role': 'tool', 'name': 'weather_search', 'content': '晴天', 'tool_call_id': '019666d90a5564c2b314d48048d9f7b0'}]}}


{'call_llm': {'messages': [AIMessage(content='北京今天天气晴朗，适合外出活动！如果有其他需要查询的城市或天气相关问题，欢迎随时告诉我~', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 264, 'prompt_tokens': 298, 'total_tokens': 562, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 239, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': None}, 'model_name': 'Qwen/QwQ-32B', 'system_fingerprint': '', 'id': '019666d96c8cd51083a77ef983476fe3', 'finish_reason': 'stop', 'logprobs': None}, id='run-bd588c19-3abd-4194-9aff-c1650775d76c-0', usag